In [7]:
#1. get xmlboxes
#2. get xmlboxes img region from bkg
#3. put the img region to new image
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path
SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
SEED_BKG_DIR = '/data/darknet/python/background/'
GENE_IMG_DIR = '/data/darknet/python/images_nanchangperson_replaced_ordinary_obj_with_bkg/'

In [8]:
def past_to_background_from_image_file(file, bboxes, background_img_array, save_name, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    time_mark = datetime.datetime.now()
    time_str = time_mark.strftime("%Y%m%d%H%M%S.%f")
    cv2.imwrite(GENE_IMG_DIR + time_str + '_' + save_name.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]
def get_bboxes_from_etree(etree):
    root = tree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        '''
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        '''
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
def generate_new_bbox(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    new_bbox = [random_x, random_x + img_obj_size[1], random_y, random_y + img_obj_size[0]]
    #print(new_bbox)
    return new_bbox
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, etree):
    new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    bboxes = get_bboxes_from_etree(etree)
    while(not check_bbox(new_bbox, bboxes)):
        print('new_bbox not suitable, retry...')
        new_bbox = generate_new_bbox(img.shape[:2], img_obj.shape[:2])
    #print('new_bbox succussful')
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, (new_bbox[0], new_bbox[2]), obj_element, tree)
    
    return new_pil_img, new_xml_etree

def check_bbox(new_bbox, bboxes):
    for bbox in bboxes:
        if(IOU(new_bbox, bbox) > 0.005):
            return False
        continue
    return True

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)

seed_xml_names.sort()
for xml_name in seed_xml_names:
    print(xml_name)       
    if(not (int(xml_name[:-4]) >= 420000)):
        continue
    background_img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    bboxes = get_bboxes_from_etree(tree)
    past_to_background_from_image_file(SEED_BKG_DIR + seed_bkg_names[0], bboxes, background_img_data, 
                                       SEED_IMG_DIR + xml_name[:-3] + 'jpg')

00000420000.xml
00000420001.xml
00000420002.xml
00000420003.xml
00000420004.xml
00000420005.xml
00000420006.xml
00000420007.xml
00000420008.xml
00000420009.xml
00000420010.xml
00000420011.xml
00000420012.xml
00000420013.xml
00000420014.xml
00000420015.xml
00000420016.xml
00000420017.xml
00000420018.xml
00000420019.xml
00000420020.xml
00000420021.xml
00000420022.xml
00000420023.xml
00000420024.xml
00000420025.xml
00000420026.xml
00000420027.xml
00000420028.xml
00000420029.xml
00000420030.xml
00000420031.xml
00000420032.xml
00000420033.xml
00000420034.xml
00000420035.xml
00000420036.xml
00000420037.xml
00000420038.xml
00000420039.xml
00000420040.xml
00000420041.xml
00000420042.xml
00000420043.xml
00000420044.xml
00000420045.xml
00000420046.xml
00000420047.xml
00000420048.xml
00000420049.xml
00000420050.xml
00000420051.xml
00000420052.xml
00000420053.xml
00000420054.xml
00000420055.xml
00000420056.xml
00000420057.xml
00000420058.xml
00000420059.xml
00000420060.xml
00000420061.xml
00000420

00000420518.xml
00000420519.xml
00000420520.xml
00000420521.xml
00000420522.xml
00000420523.xml
00000420524.xml
00000420525.xml
00000420526.xml
00000420527.xml
00000420528.xml
00000420529.xml
00000420530.xml
00000420531.xml
00000420532.xml
00000420533.xml
00000420534.xml
00000420535.xml
00000420536.xml
00000420537.xml
00000420538.xml
00000420539.xml
00000420540.xml
00000420541.xml
00000420542.xml
00000420543.xml
00000420544.xml
00000420545.xml
00000420546.xml
00000420547.xml
00000420548.xml
00000420549.xml
00000420550.xml
00000420551.xml
00000420552.xml
00000420553.xml
00000420554.xml
00000420555.xml
00000420556.xml
00000420557.xml
00000420558.xml
00000420559.xml
00000420560.xml
00000420561.xml
00000420562.xml
00000420563.xml
00000420564.xml
00000420565.xml
00000420566.xml
00000420567.xml
00000420568.xml
00000420569.xml
00000420570.xml
00000420571.xml
00000420572.xml
00000420573.xml
00000420574.xml
00000420575.xml
00000420576.xml
00000420577.xml
00000420578.xml
00000420579.xml
00000420

00000421031.xml
00000421032.xml
00000421033.xml
00000421034.xml
00000421035.xml
00000421036.xml
00000421037.xml
00000421038.xml
00000421039.xml
00000421040.xml
00000421041.xml
00000421042.xml
00000421043.xml
00000421044.xml
00000421045.xml
00000421046.xml
00000421047.xml
00000421048.xml
00000421049.xml
00000421050.xml
00000421051.xml
00000421052.xml
00000421053.xml
00000421054.xml
00000421055.xml
00000421056.xml
00000421057.xml
00000421058.xml
00000421059.xml
00000421060.xml
00000421061.xml
00000421062.xml
00000421063.xml
00000421064.xml
00000421065.xml
00000421066.xml
00000421067.xml
00000421068.xml
00000421069.xml
00000421070.xml
00000421071.xml
00000421072.xml
00000421073.xml
00000421074.xml
00000421075.xml
00000421076.xml
00000421077.xml
00000421078.xml
00000421079.xml
00000421080.xml
00000421081.xml
00000421082.xml
00000421083.xml
00000421084.xml
00000421085.xml
00000421086.xml
00000421087.xml
00000421088.xml
00000421089.xml
00000421090.xml
00000421091.xml
00000421092.xml
00000421